In [2]:
import requests
import time
import json

from inspirehep import *

In [3]:
f=open('data/db_LA.json','r')
db=json.load(f)
f.close()
idsLA=[d.get('LA_institution_id') for d in db]
idsAU=[d.get('author_id') for d in db]
db_index=get_index(db)

In [4]:
db_inst_index=get_index(db,index='LA_institution_id')

In [63]:
collaborations_old==collaborations

True

Pagination

In [57]:
#bkp_col=collaborations
collaborations=collaborations[:-6]

In [75]:
collaborations

['STAR',
 'MicroBooNE',
 'CLAS',
 'DUNE',
 'NA62',
 'PHENIX',
 'BESIII',
 'DES',
 'Super-Kamiokande',
 'LIGO Scientific',
 'KM3NeT',
 'HAWC',
 'Pierre Auger',
 'IceCube',
 'CTA Observatory',
 'Telescope Array',
 'MAGIC',
 'SuperCDMS',
 'CUPID',
 'CUORE',
 'KATRIN',
 'REDTOP',
 'KAGRA',
 'Muon Collider',
 'Mu2e-II',
 'ILC International Development Team',
 'Daya Bay',
 'Virgo',
 'NEMO-3',
 'ATLAS',
 'Euclid',
 'LHCb',
 'CMS',
 'ANTARES',
 'GERDA',
 'HADES',
 'n_TOF',
 'PANDA',
 'Jefferson Lab Hall A',
 'LUX-ZEPLIN',
 'Belle',
 'LHAASO',
 'H.E.S.S.',
 'JUNO',
 'AMS',
 'XENON',
 'SHiP',
 'ALICE',
 'NA61/SHINE',
 'DAMPE',
 'H1',
 'NOvA',
 'CTA LST Project',
 'OPERA',
 'COMPASS',
 'GlueX',
 'LEGEND',
 'Event Horizon Telescope',
 'MPD',
 'LiteBIRD',
 'Double Chooz',
 'Fermi-LAT',
 'nEXO',
 'SDSS-IV',
 'NEXT',
 'CREX',
 'BaBar',
 'CDF',
 'DarkSide',
 'EXO-200',
 'Tibet ASγ',
 'SPT-3G',
 'Mice',
 'CTA Consortium',
 'SNO+',
 'CSNS Back-n',
 'WA105',
 'NA48/2',
 'Muon g-2',
 'DEAP',
 'T2K',
 'Cha

In [77]:
page_max=12

In [80]:
collaborations

['STAR',
 'MicroBooNE',
 'CLAS',
 'DUNE',
 'NA62',
 'PHENIX',
 'BESIII',
 'DES',
 'Super-Kamiokande',
 'LIGO Scientific',
 'KM3NeT',
 'HAWC',
 'Pierre Auger',
 'IceCube',
 'CTA Observatory',
 'Telescope Array',
 'MAGIC',
 'SuperCDMS',
 'CUPID',
 'CUORE',
 'KATRIN',
 'REDTOP',
 'KAGRA',
 'Muon Collider',
 'Mu2e-II',
 'ILC International Development Team',
 'Daya Bay',
 'Virgo',
 'NEMO-3',
 'ATLAS',
 'Euclid',
 'LHCb',
 'CMS',
 'ANTARES',
 'GERDA',
 'HADES',
 'n_TOF',
 'PANDA',
 'Jefferson Lab Hall A',
 'LUX-ZEPLIN',
 'Belle',
 'LHAASO',
 'H.E.S.S.',
 'JUNO',
 'AMS',
 'XENON',
 'SHiP',
 'ALICE',
 'NA61/SHINE',
 'DAMPE',
 'H1',
 'NOvA',
 'CTA LST Project',
 'OPERA',
 'COMPASS',
 'GlueX',
 'LEGEND',
 'Event Horizon Telescope',
 'MPD',
 'LiteBIRD',
 'Double Chooz',
 'Fermi-LAT',
 'nEXO',
 'SDSS-IV',
 'NEXT',
 'CREX',
 'BaBar',
 'CDF',
 'DarkSide',
 'EXO-200',
 'Tibet ASγ',
 'SPT-3G',
 'Mice',
 'CTA Consortium',
 'SNO+',
 'CSNS Back-n',
 'WA105',
 'NA48/2',
 'Muon g-2',
 'DEAP',
 'T2K',
 'Cha

In [ ]:
#page_max=float('inf')
page=0
emergency_exit=300
mp=0
LONG_URL=False
while page>=0:
    print(page_max)
    if page_max<13 or LONG_URL: #Only few pages left or too long URL. Go through all of them
        #collaborations=collaborations[:102]
        collaborations_old=collaborations #last 10 pages
    if mp>=emergency_exit:
        print('Emergency Exit!')
        break
    mp=mp+1
    if len(collaborations)>0 and  collaborations_old!=collaborations:
        cn_search='https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=ac>100+and+de>2018'
        cn=f"{cn_search}+and+not+cn+{'+and+not+cn+'.join(collaborations)}".replace(' ','+')
        collaborations_old=collaborations
        page=0
        ir len(cn)>2080:
            LONG_URL=True
    else:
        #increase pagination
        if len(collaborations)==0:
            extra=''
        else:
            extra='+and+not+cn+'
        page=page+1
        cn_search=f'https://inspirehep.net/api/literature?sort=mostrecent&size=25&page={page}&q=ac>100+and+de>2018'
        cn=f"{cn_search}{extra}{'+and+not+cn+'.join(collaborations[:102])}".replace(' ','+')

    print(cn)
    time.sleep(sleep)
    r=requests.get(cn)
    d=r.json()
    page_max=int(d.get('hits').get('total')/25)+1
    if page>page_max:
        break

    l=d.get('hits').get('hits')

    #Not affilation in `'authors'` list for collaborations!

    for ld in l:
        lit=ld.get('metadata')
        if not lit.get('collaborations'):
            continue # next paper
        try:
            collaboration=lit.get('collaborations')[0].get('value')
        except:
            collaboration=''

        if collaboration in collaborations:
            continue #next paper

        print(f'collaboration → {collaboration}')
        la=lit.get('authors')
        ii=0
        for a in la:
            FoS=[collaboration]
            try:
                PAC=lit['primary_arxiv_category']
            except:
                PAC=[]
            
            print(ii,end='\r')
            ii=ii+1
            CONTINUE,author_id,url_author,FoS,PAC,db=get_work(db,a,idsLA,idsAU,db_index,FoS,PAC)
            if CONTINUE:
                continue #next author
            positions,email_addresses,name,aids=get_author(url_author) #USES API!
            try:
                inst_id=[d.get('record').get('$ref').split('/')[-1] 
                     for d in positions if d.get('record') and d.get('current')][0]
            except:
                inst_id=None
            if inst_id not in idsLA:
                continue #next author
            di=db[db_inst_index[inst_id]]

            new={'author_id':author_id,
                     'name':name,
                     'LA_institution_id':inst_id,
                     'external_system_identifiers':di.get('external_system_identifiers'),
                     'LA_country':di.get('LA_country'),
                     'LA_institution':di.get('LA_institution'),
                     'email_addresses':email_addresses,
                     'positions':positions,
                     'ids':aids,
                     'primary_arxiv_category':PAC,
                     'Fields of Study': FoS # in ['Particle physics','Cosmology','Astrophysics','Mathematical physics']+collaborations
                    }
            db.append(new)
            #raise Exception('STOP')            
            #break
        collaborations.append(collaboration) #Only after search for all the authors
        f=open('db_LA.json','w')
        json.dump(db,f)
        f.close()    
        #==============
        #break            
        

12
https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=5&q=ac>100+and+de>2018+and+not+cn+STAR+and+not+cn+MicroBooNE+and+not+cn+CLAS+and+not+cn+DUNE+and+not+cn+NA62+and+not+cn+PHENIX+and+not+cn+BESIII+and+not+cn+DES+and+not+cn+Super-Kamiokande+and+not+cn+LIGO+Scientific+and+not+cn+KM3NeT+and+not+cn+HAWC+and+not+cn+Pierre+Auger+and+not+cn+IceCube+and+not+cn+CTA+Observatory+and+not+cn+Telescope+Array+and+not+cn+MAGIC+and+not+cn+SuperCDMS+and+not+cn+CUPID+and+not+cn+CUORE+and+not+cn+KATRIN+and+not+cn+REDTOP+and+not+cn+KAGRA+and+not+cn+Muon+Collider+and+not+cn+Mu2e-II+and+not+cn+ILC+International+Development+Team+and+not+cn+Daya+Bay+and+not+cn+Virgo+and+not+cn+NEMO-3+and+not+cn+ATLAS+and+not+cn+Euclid+and+not+cn+LHCb+and+not+cn+CMS+and+not+cn+ANTARES+and+not+cn+GERDA+and+not+cn+HADES+and+not+cn+n_TOF+and+not+cn+PANDA+and+not+cn+Jefferson+Lab+Hall+A+and+not+cn+LUX-ZEPLIN+and+not+cn+Belle+and+not+cn+LHAASO+and+not+cn+H.E.S.S.+and+not+cn+JUNO+and+not+cn+AMS+and+not+cn+XENON+and+

In [83]:
collaborations_old==collaborations

True

In [82]:
cn

'https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=5&q=ac>100+and+de>2018+and+not+cn+STAR+and+not+cn+MicroBooNE+and+not+cn+CLAS+and+not+cn+DUNE+and+not+cn+NA62+and+not+cn+PHENIX+and+not+cn+BESIII+and+not+cn+DES+and+not+cn+Super-Kamiokande+and+not+cn+LIGO+Scientific+and+not+cn+KM3NeT+and+not+cn+HAWC+and+not+cn+Pierre+Auger+and+not+cn+IceCube+and+not+cn+CTA+Observatory+and+not+cn+Telescope+Array+and+not+cn+MAGIC+and+not+cn+SuperCDMS+and+not+cn+CUPID+and+not+cn+CUORE+and+not+cn+KATRIN+and+not+cn+REDTOP+and+not+cn+KAGRA+and+not+cn+Muon+Collider+and+not+cn+Mu2e-II+and+not+cn+ILC+International+Development+Team+and+not+cn+Daya+Bay+and+not+cn+Virgo+and+not+cn+NEMO-3+and+not+cn+ATLAS+and+not+cn+Euclid+and+not+cn+LHCb+and+not+cn+CMS+and+not+cn+ANTARES+and+not+cn+GERDA+and+not+cn+HADES+and+not+cn+n_TOF+and+not+cn+PANDA+and+not+cn+Jefferson+Lab+Hall+A+and+not+cn+LUX-ZEPLIN+and+not+cn+Belle+and+not+cn+LHAASO+and+not+cn+H.E.S.S.+and+not+cn+JUNO+and+not+cn+AMS+and+not+cn+XENON+and+no

In [33]:
collaborations=collaborations[:-1]

In [13]:
collaborations=['STAR',
 'MicroBooNE',
 'CLAS',
 'DUNE',
 'NA62',
 'PHENIX',
 'BESIII',
 'DES',
 'Super-Kamiokande',
 'LIGO Scientific',
 'KM3NeT',
 'HAWC',
 'Pierre Auger',
 'IceCube',
 'CTA Observatory',
 'Telescope Array',
 'MAGIC',
 'SuperCDMS',
 'CUPID',
 'CUORE',
 'KATRIN',
 'REDTOP',
 'KAGRA',
 'Muon Collider',
 'Mu2e-II',
 'ILC International Development Team',
 'Daya Bay',
 'Virgo',
 'NEMO-3',
 'ATLAS',
 'Euclid',
 'LHCb',
 'CMS',
 'ANTARES',
 'GERDA',
 'HADES',
 'n_TOF',
 'PANDA',
 'Jefferson Lab Hall A',
 'LUX-ZEPLIN',
 'Belle',
 'LHAASO',
 'H.E.S.S.',
 'JUNO',
 'AMS',
 'XENON',
 'SHiP',
 'ALICE',
 'NA61/SHINE',
 'DAMPE',
 'H1',
 'NOvA',
 'CTA LST Project',
 'OPERA',
 'COMPASS',
 'GlueX',
 'LEGEND',
 'Event Horizon Telescope']

In [15]:
collaborations_old=[]

In [6]:
author_id='991245'
db[db_index[author_id]]

{'author_id': '991245',
 'name': {'value': 'Rosenfeld, Rogerio',
  'preferred_name': 'Rogerio Rosenfeld'},
 'LA_institution_id': '1373228',
 'external_system_identifiers': [],
 'LA_country': 'Brazil',
 'LA_institution': 'LIneA, Rio de Janeiro',
 'email_addresses': [{'value': 'rosenfel@ift.unesp.br', 'current': True}],
 'positions': [{'record': {'$ref': 'https://inspirehep.net/api/institutions/1373228'},
   'current': True,
   'institution': 'LIneA, Rio de Janeiro',
   'curated_relation': True},
  {'rank': 'SENIOR',
   'record': {'$ref': 'https://inspirehep.net/api/institutions/903185'},
   'current': True,
   'institution': 'Sao Paulo, IFT',
   'curated_relation': True},
  {'record': {'$ref': 'https://inspirehep.net/api/institutions/903081'},
   'end_date': '1994',
   'start_date': '1992',
   'institution': 'Northeastern U.',
   'curated_relation': True},
  {'rank': 'POSTDOC',
   'record': {'$ref': 'https://inspirehep.net/api/institutions/903303'},
   'end_date': '1992',
   'start_date

In [62]:
CONTINUE,author_id,url_author,FoS,PAC

(False,
 '1282520',
 'https://inspirehep.net/api/authors/1282520',
 ['ANTARES'],
 ['astro-ph.HE'])

In [73]:
import pandas as pd

In [74]:
df=pd.DataFrame(db)

In [81]:
df[df.name.str['value'].str.contains('Acero')]

,author_id,name,LA_institution_id,external_system_identifiers,LA_country,LA_institution,email_addresses,positions,ids,primary_arxiv_category,Fields of Study
1176,1023526,"{'value': 'Acero Ortega, Mario Andres', 'name_...",1128047,"[{'value': 'grid.441871.f', 'schema': 'GRID'},...",Colombia,"U. Atlantico, Barranquilla",[{'value': 'marioacero@mail.uniatlantico.edu.c...,"[{'rank': 'SENIOR', 'record': {'$ref': 'https:...","[{'value': '0000-0002-0835-0641', 'schema': 'O...","[gr-qc, hep-th, physics.ins-det, hep-ph]","[Cosmology, Mathematical physics, DUNE, Partic..."


In [1]:
import json

In [2]:
f=open('data/db_LA.json','r')
df=json.load(f)
f.close()

In [4]:
f=open('db_LA.json','r')
dfn=json.load(f)
f.close()

In [14]:
df[''primary_arxiv_category']

{'author_id': '1876618',
 'name': {'value': 'Ramírez, Valeria'},
 'LA_institution_id': '909015',
 'external_system_identifiers': [{'value': 'grid.412866.f', 'schema': 'GRID'},
  {'value': 'https://ror.org/031f8kt38', 'schema': 'ROR'},
  {'value': 'INST-77020', 'schema': 'SPIRES'}],
 'LA_country': 'Mexico',
 'LA_institution': 'UAEH, Pachuca',
 'email_addresses': [],
 'positions': [],
 'ids': [{'value': 'V.Ramirez.1', 'schema': 'INSPIRE BAI'}],
 'primary_arxiv_category': ['gr-qc'],
 'Fields of Study': ['Mathematical physics']}

In [21]:
[d.get('primary_arxiv_category') for d in df][30:40]

[['nlin.AO',
  'cond-mat.str-el',
  'cond-mat.other',
  'hep-ex',
  'hep-ph',
  'astro-ph.GA',
  'physics.gen-ph',
  'hep-th',
  'astro-ph.HE',
  'hep-lat',
  'quant-ph',
  'math-ph',
  'gr-qc',
  'cond-mat.stat-mech',
  'astro-ph.CO',
  'astro-ph.SR',
  'nucl-th',
  'physics.pop-ph'],
 ['cond-mat.stat-mech',
  'cond-mat.str-el',
  'hep-ex',
  'hep-ph',
  'hep-th',
  'quant-ph',
  'gr-qc',
  'math-ph'],
 ['math-ph', 'hep-th'],
 ['math-ph', 'hep-th'],
 ['hep-ph'],
 ['hep-ph'],
 ['hep-ph'],
 ['hep-ph', 'astro-ph.CO'],
 ['hep-ph'],
 ['hep-th']]

In [20]:
[d.get('primary_arxiv_category') for d in dfn][30:40]

[['hep-th', 'hep-ph', 'gr-qc'],
 ['hep-th', 'hep-ph', 'gr-qc'],
 ['hep-th'],
 ['hep-th'],
 ['hep-ph'],
 ['hep-ph'],
 ['hep-ph'],
 ['astro-ph.CO', 'hep-ph'],
 ['hep-ph'],
 ['hep-th']]